Use this notebook to make API calls to Ticketmaster

In [210]:
# dependencies
import requests
import json
import pandas as pd
import time

In [227]:
# API keys
from config import ticketmaster_api_key as tm_key

### Find Ticketmaster events

In [76]:
# Info for the Ticketmaster Discovery API:
## https://developer.ticketmaster.com/products-and-docs/apis/discovery-api/v2/

# To find searchable values and IDs for different query types, use:
## https://developer.ticketmaster.com/api-explorer/v2/

# Ticketmaster URL
tm_root_url = "https://app.ticketmaster.com/discovery/v2/"

In [234]:
# test run: searching for all events in Berkeley, CA

# set up the query parameters
event_params = {"classificationName": "Music"
               ,"city": "Berkeley"
               ,"stateCode": "CA"
               ,"countryCode": "US"
               }


In [243]:
# function to make an API call to Ticketmaster to search for events
# input a library of desired query parameters for an event search
# https://developer.ticketmaster.com/products-and-docs/apis/discovery-api/v2/#search-events-v2


def get_tm_events(query):
    
    events_string = 'events?'
    query_string = ''
    
    # begin constructing a URL
    for i in range(len(query)):
        
        # save the parameter key
        key_string = str(list(query.items())[i][0]) 
        
        # save the parameter value
        value_string = str(list(query.items())[i][1])
        
        # concatenate parameter keys, values in API call format
        query_string += key_string + '=' + value_string + '&'    

    # finalize the query URL
    tm_url = tm_root_url + events_string + query_string + 'apikey=' + tm_key

    # make the API call with the completed URL
    events_data = requests.get(tm_url)
    events_json = events_data.json()
    
    return events_json

In [260]:
# function to return multiple pages from an 'events' API call to Ticketmaster
# input a library of desired query parameters for an event search
# https://developer.ticketmaster.com/products-and-docs/apis/discovery-api/v2/#search-events-v2

def get_tm_event_pages(query):
    
    query['size'] = '20'      # set a standard page size
    events_string = 'events?'
    query_start = ''
    
    # begin constructing a URL
    for i in range(len(query)):
        
        # save the parameter key
        key_string = str(list(query.items())[i][0]) 
        
        # save the parameter value
        value_string = str(list(query.items())[i][1])
        
        # concatenate parameter keys, values in API call format
        query_start += key_string + '=' + value_string + '&'
    
    # empty list to save outputs from each 'page' output by the API call
    pages = []
    
    # make API calls with finalized query URLs for each page
    for page_num in range(50):
        
        # individual query for each page
        query_string = query_start + f"&page={page_num}&"
        
        # finalized query URL 
        query_url = tm_root_url + events_string + query_string + 'apikey=' + tm_key
        
        # make the API call to retrieve data in json format
        query_data = requests.get(query_url).json()
        
        # append the query output to a list
        pages.append(query_data)
        
        # wait 0.20 sec until the next API call (Ticketmaster only allows 5 per second)
        time.sleep(0.20)
    
    # save the list of datasets
    return pages

In [248]:
# simple API call: get events playing in Berkeley, CA
berkeley_events = get_tm_events(event_params)
print(json.dumps(berkeley_events, indent=4, sort_keys=True))

{
    "_embedded": {
        "events": [
            {
                "_embedded": {
                    "attractions": [
                        {
                            "_links": {
                                "self": {
                                    "href": "/discovery/v2/attractions/K8vZ9172pOV?locale=en-us"
                                }
                            },
                            "classifications": [
                                {
                                    "family": false,
                                    "genre": {
                                        "id": "KnvZfZ7vAev",
                                        "name": "Pop"
                                    },
                                    "primary": true,
                                    "segment": {
                                        "id": "KZFzniwnSyZfZ7v7nJ",
                                        "name": "Music"
                                    },
     

In [256]:
# hone in on actual events data
print(json.dumps(berkeley_events["_embedded"]["events"], indent=4, sort_keys=True))

[
    {
        "_embedded": {
            "attractions": [
                {
                    "_links": {
                        "self": {
                            "href": "/discovery/v2/attractions/K8vZ9172pOV?locale=en-us"
                        }
                    },
                    "classifications": [
                        {
                            "family": false,
                            "genre": {
                                "id": "KnvZfZ7vAev",
                                "name": "Pop"
                            },
                            "primary": true,
                            "segment": {
                                "id": "KZFzniwnSyZfZ7v7nJ",
                                "name": "Music"
                            },
                            "subGenre": {
                                "id": "KZazBEonSMnZfZ7vk1t",
                                "name": "Pop"
                            },
                            "sub

In [250]:
# save events data as a dataframe
berkeley_df = pd.DataFrame.from_dict(berkeley_events['_embedded']['events'])
print(berkeley_df.columns)
berkeley_df.head()

Index(['name', 'type', 'id', 'test', 'url', 'locale', 'images', 'sales',
       'dates', 'classifications', 'promoter', 'promoters', 'info',
       'pleaseNote', 'priceRanges', 'seatmap', 'accessibility', 'ticketLimit',
       'ageRestrictions', 'ticketing', '_links', '_embedded', 'outlets'],
      dtype='object')


,name,type,id,test,url,locale,images,sales,dates,classifications,...,pleaseNote,priceRanges,seatmap,accessibility,ticketLimit,ageRestrictions,ticketing,_links,_embedded,outlets
0,Carly Rae Jepsen: The So Nice Tour,event,G5vYZ9G2CSEhV,False,https://www.ticketmaster.com/carly-rae-jepsen-...,en-us,"[{'ratio': '3_2', 'url': 'https://s1.ticketm.n...",{'public': {'startDateTime': '2022-06-10T17:00...,"{'start': {'localDate': '2022-10-21', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...",...,COVID Warning: An inherent risk of exposure to...,"[{'type': 'standard', 'currency': 'USD', 'min'...",{'staticUrl': 'https://maps.ticketmaster.com/m...,{'info': 'The venue can accommodate wheelchair...,{'info': 'There is an 8 ticket limit per house...,{'legalAgeEnforced': False},{'safeTix': {'enabled': True}},{'self': {'href': '/discovery/v2/events/G5vYZ9...,{'venues': [{'name': 'Greek Theatre-U.C. Berke...,NaN
1,Baek Yerin,event,Z7r9jZ1Adbp-y,False,https://www.ticketmaster.com/event/Z7r9jZ1Adbp-y,en-us,"[{'ratio': '3_2', 'url': 'https://s1.ticketm.n...",{'public': {'startDateTime': '2022-09-22T17:00...,"{'start': {'localDate': '2022-12-07', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'self': {'href': '/discovery/v2/events/Z7r9jZ...,"{'venues': [{'name': 'The UC Theatre', 'type':...","[{'url': 'https://www.theuctheatre.org/', 'typ..."
2,Papadosio,event,Z7r9jZ1Ad4PPJ,False,https://www.ticketmaster.com/event/Z7r9jZ1Ad4PPJ,en-us,"[{'ratio': '16_9', 'url': 'https://s1.ticketm....",{'public': {'startDateTime': '1900-01-01T06:00...,"{'start': {'localDate': '2022-10-15', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'self': {'href': '/discovery/v2/events/Z7r9jZ...,"{'venues': [{'name': 'The UC Theatre', 'type':...","[{'url': 'https://www.theuctheatre.org/', 'typ..."
3,Real Friends,event,Z7r9jZ1Ad4A7o,False,https://www.ticketmaster.com/event/Z7r9jZ1Ad4A7o,en-us,"[{'ratio': '16_9', 'url': 'https://s1.ticketm....",{'public': {'startDateTime': '1900-01-01T06:00...,"{'start': {'localDate': '2022-10-23', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...",...,NaN,NaN,{'staticUrl': 'https://content.resale.ticketma...,NaN,NaN,NaN,NaN,{'self': {'href': '/discovery/v2/events/Z7r9jZ...,"{'venues': [{'name': 'Cornerstone - CA', 'type...",[{'url': 'https://www.cornerstoneberkeley.com/...
4,OMEGA X,event,rZ7HnEZ1A389-g,False,https://www.ticketweb.com/event/omega-x-the-uc...,en-us,"[{'ratio': '3_2', 'url': 'https://s1.ticketm.n...",{'public': {'startDateTime': '2022-08-18T21:00...,{'access': {'startDateTime': '2022-10-22T01:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...",...,ALL SALES FINAL. NO REFUNDS/EXCHANGES. COVID-1...,"[{'type': 'standard', 'currency': 'USD', 'min'...",NaN,NaN,NaN,NaN,NaN,{'self': {'href': '/discovery/v2/events/rZ7HnE...,"{'venues': [{'name': 'The UC Theatre', 'type':...",NaN


In [261]:
# larger API call: get multiple pages of events playing in Berkeley, CA
berkeley_events_pages = get_tm_event_pages(event_params)
print(json.dumps(berkeley_events_pages, indent=4, sort_keys=True))

[
    {
        "_embedded": {
            "events": [
                {
                    "_embedded": {
                        "attractions": [
                            {
                                "_links": {
                                    "self": {
                                        "href": "/discovery/v2/attractions/K8vZ9172pOV?locale=en-us"
                                    }
                                },
                                "classifications": [
                                    {
                                        "family": false,
                                        "genre": {
                                            "id": "KnvZfZ7vAev",
                                            "name": "Pop"
                                        },
                                        "primary": true,
                                        "segment": {
                                            "id": "KZFzniwnSyZfZ7v7nJ",
          

In [262]:
len(berkeley_events_pages)

50